In [ ]:
## Data Ingestion
from src.data.ingestion import DataIngestion
train, test = DataIngestion(mode="dev").execute()

In [ ]:
## Feature Engineering
from src.features.engineering import FeatureEngineering
train = FeatureEngineering(train).execute()
test = FeatureEngineering(test).execute()

In [ ]:
# Drop Useless Columns
def drop_cols(df):
    df.drop(
        [
            "ID",
            "UpsellDate",
            "PaymentMethod",
            "TransactionDates",
            "PaymentsHistory",
            "SupplierName",
            "Town",
            "RegistrationDateParsed",
            "ExpectedTermDateParsed",
            "FirstPaymentDateParsed",
            "LastPaymentDateParsed"
        ],
        inplace=True,
        axis=1
    )
drop_cols(train)
drop_cols(test)
train.head()
test.head()

In [ ]:
# Feature Encoding



In [ ]:
# Feature Scaling

In [ ]:
# Approach two modelling

In [ ]:
## Inference: model, X  -> yPred

In [ ]:
# Calculation of Metric (RMSE)

In [ ]:
# Outlier detection (if any)

# TASKS
# Tasks
    - Feature selection  Aum
        - Automated selection based on Genie Index

    - Pipeline setup  Nikhil
        - Pre-process
            - ingestion (DONE)
            - Features (DONE)
            - Encoding (Pending)
            - scaling (if applicable)
        - Modelling
            - Approach2
                - Sliding updates for features for Train and test
        
        - validation
            - Calculate RMSE
